<h1>MassBalanceMachine Data Processing - Example for Retrieving Training Features for the Iceland Region (WGMS)</h1>
<p style='text-align: justify;'>
In this notebook, we will demonstrate the data processing workflow of the MassBalanceMachine using an example with glaciological data from Icelandic glaciers. This example will guide you through the data processing pipeline, which retrieves topographical and meteorological features for all points with glaciological surface mass balance observations, and transforms the data to a monthly resolution.
</p>
<p style='text-align: justify;'>
We begin by importing some basic libraries along with the <code>massbalancemachine</code> library. Next, we specify the location where we will store the files for the region of interest . The data used in this example is from the <a href='https://icelandicglaciers.is/index.html#/page/map'>Icelandic Glaciers inventory</a>, that was already preprocessed, to align with the WGMS data format we are using, in this <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/data_preprocessing.ipynb'>notebook</a>. You can also download data from the <a href='https://wgms.ch/data_databaseversions/'>WGMS database</a>, as you desire.
</p>

<b>Note:</b> WGMS data can contain errors or incorrect data, please check the data for correctness before using it.   

<b>Note:</b> All data, stake measurements and glaciers outlines, are expected to have the WGS84 CRS.

In [1]:
import pandas as pd
import geopandas as gpd

import massbalancemachine as mbm
example = False
thomas = True
WGMS = False
era5carra = True # False if CARRA

<h2>1. Load your Target Surface Mass Balance Dataset and Retrieve RGI ID per Stake</h2>
<p style='text-align: justify;'>
In this step, we define and load our glaciological data from a region of interest. The WGMS dataset does not include RGI IDs by default, so we need to retrieve them from a glacier outline shapefile provided by the Randolph Glacier Inventory (v6). Each stake is then matched with an RGI ID. The RGI ID is necessary for the MassBalanceMachine to add additional topographical and meteorological features for training stage.
</p>
<p style='text-align: justify;'>
<b>How to Retrieve the Glacier Outlines:</b> Download the shapefiles for the region of interest from this <a href='https://daacdata.apps.nsidc.org/pub/DATASETS/nsidc0770_rgi_v6/'>link</a>. Extract the files and copy the .shp, .prj, .shx, and .dbf files in the correct directory so that you can use it with the Jupyter Notebook. Also, make sure you point to the correct directory and files in the next code cell.
</p>

<b>Note:</b> Data records that have an invalid FROM_DATE or TO_DATE, where the day is indicated as 99, are deleted from the dataset.

In [2]:
if example:
	# Specify the filename of the input file with the raw data
	target_data_fname = './example_data/iceland/files/iceland_wgms_dataset.csv'
	# Specify the shape filename of the glaciers outline obtained from RGIv6
	glacier_outline_fname = './example_data/iceland/glacier_outlines/06_rgi60_Iceland.shp'

	# Load the target data and the glacier outlines
	data = pd.read_csv(target_data_fname)
	glacier_outline = gpd.read_file(glacier_outline_fname)

<h3>1.1 Match the Stake Measurements with a RGI ID</h3>
<p style='text-align: justify;'>
Based on the location of the stake measurement given by POINT_LAT and POINT_LON, each data record is matched with the RGI ID for the glacier where the stake is located.
</p>

In [3]:
if example:
	# Get the RGI ID for each stake measurement for the region of interest
	data = mbm.data_processing.utils.get_rgi(data=data, glacier_outlines=glacier_outline)

<p style='text-align: justify;'>
Then, we can create a MassBalanceMachine `Dataset`, by using the loaded dataframe for Norway stake data together with the matched RGI IDs, as such: 
</p>

In [4]:
if example:
	# Provide the column name for the column that has the RGI IDs for each of the stakes
	dataset = mbm.Dataset(data=data, region_name='iceland', data_path='./example_data/iceland/files/')
elif thomas:
	data = pd.read_csv('./example_data/svalbard/Svalbard_seasonalMB.csv')
	# If using CARRA data, remove year before 1990,9,1
	#if era5carra == False:
	data = data.loc[data['YEAR'] > 1990]
	data.reset_index(inplace=True)
	
	dataset = mbm.Dataset(data=data, region_name='svalbard', data_path='./example_data/svalbard/')


elif WGMS:
	data = pd.read_csv('./example_data/svalbard/WGMS_mass_balance_point_RGI07.csv')
	# Fix data types
	data['RGIId'] = data.rgi
	data["YEAR"]=data["YEAR"].values.astype('int64')
	data["FROM_DATE"]=data["FROM_DATE"].values.astype('str')
	data["TO_DATE"]=data["TO_DATE"].values.astype('str')

	# Fix unknown dates
	data.loc[data['BALANCE_CODE']== 'BS',"FROM_DATE"]=data.loc[data['BALANCE_CODE']== 'BS',"FROM_DATE"].str.replace('9999$', '0501', regex=True)
	data.loc[data['BALANCE_CODE']== 'BW',"FROM_DATE"]=data.loc[data['BALANCE_CODE']== 'BW',"FROM_DATE"].str.replace('9999$', '1001', regex=True)
	data.loc[data['BALANCE_CODE']== 'BA',"FROM_DATE"]=data.loc[data['BALANCE_CODE']== 'BA',"FROM_DATE"].str.replace('9999$', '1001', regex=True)
	data.loc[data['BALANCE_CODE']== 'BS',"TO_DATE"]=data.loc[data['BALANCE_CODE']== 'BS',"TO_DATE"].str.replace('9999$', '0930', regex=True)
	data.loc[data['BALANCE_CODE']== 'BW',"TO_DATE"]=data.loc[data['BALANCE_CODE']== 'BW',"TO_DATE"].str.replace('9999$', '0430', regex=True)
	data.loc[data['BALANCE_CODE']== 'BA',"TO_DATE"]=data.loc[data['BALANCE_CODE']== 'BA',"TO_DATE"].str.replace('9999$', '0930', regex=True)

	# Remove empty coordinates
	data = data.loc[~data['POINT_LAT'].isna()]
	data.reset_index(inplace=True)


In [5]:
if WGMS:
	# Remove BA if BS and BW are available
	data["id"] = data.groupby(data[["YEAR", "POINT_LAT"]].apply(frozenset, axis=1)).ngroup() + 1
	for i in data['id'].unique():
		codes = data.loc[data['id'] == i,'BALANCE_CODE']
		if len(codes) == 3:
			data = data.drop(codes.index[codes == 'BA'])

		if (len(codes) == 2) and ('BA' in codes.values):
			if 'BS' in codes.values:
				data.loc[codes.index[codes == 'BA'],"POINT_BALANCE"] = data.loc[codes.index[codes == 'BA'],"POINT_BALANCE"].values - data.loc[codes.index[codes == 'BS'],"POINT_BALANCE"].values
				data.loc[codes.index[codes == 'BA'],"TO_DATE"] = data.loc[codes.index[codes == 'BA'],"YEAR"].astype('str').values+'0430'
				data.loc[codes.index[codes == 'BA'],"BALANCE_CODE"] = 'BW'
			if 'BW' in codes.values:
				data.loc[codes.index[codes == 'BA'],"POINT_BALANCE"] = data.loc[codes.index[codes == 'BA'],"POINT_BALANCE"].values - data.loc[codes.index[codes == 'BW'],"POINT_BALANCE"].values
				data.loc[codes.index[codes == 'BA'],"FROM_DATE"] = data.loc[codes.index[codes == 'BA'],"YEAR"].astype('str').values+'0501'
				data.loc[codes.index[codes == 'BA'],"BALANCE_CODE"] = 'BS'
	data.reset_index(inplace=True)

	# Provide the column name for the column that has the RGI IDs for each of the stakes
	dataset = mbm.Dataset(data=data, region_name='svalbard', data_path='./example_data/svalbard/')


<h2>2. Get Topographical Features per Stake Measurement</h2>
<p style='text-align: justify;'>
Once we have created a <code>Dataset</code>, the first thing we can do is to add topographical data in our dataset. This can be done automatically with the MassBalanceMachine (which calls OGGM) by doing the following:
</p>

In [6]:
# Specify the topographical features of interest 
# Please see the OGGM documentation what variables are available: https://oggm.org/tutorials/stable/notebooks/10minutes/machine_learning.html ('topo', 'slope_factor', 'dis_from_border')
voi_topographical = ['aspect', 'slope']

# Retrieve the topographical features for each stake measurement and add them to the dataset
dataset.get_topo_features(vois=voi_topographical)

#dataset.data.to_csv('/mnt/c/IBDV/datasetbeforeclimate_4Kamilla.csv')
#display(dataset.data)

2024-10-09 15:24:40: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2024-10-09 15:24:40: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2024-10-09 15:24:40: oggm.cfg: Multiprocessing: using all available processors (N=256)
2024-10-09 15:24:42: oggm.cfg: PARAMS['border'] changed from `80` to `10`.
2024-10-09 15:24:42: oggm.cfg: Multiprocessing switched ON after user settings.
2024-10-09 15:24:42: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2024-10-09 15:24:42: oggm.workflow: init_glacier_directories from prepro level 3 on 10 glaciers.
2024-10-09 15:24:42: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 10 glaciers
2024-10-09 15:24:47: oggm.workflow: Execute entity tasks [gridded_attributes] on 10 glaciers


<h2>3. Get Meteorological Features per Stake Measurement</h2>
<p style='text-align: justify;'>
Once we have the topographical data, we can add the necessary climate data for the dataset. This is done by pulling that from ERA5-Land database for the region of interest. Before the climate data is matched with the stake measurements, we need to manually download the climate data for the region of interest from <a href='https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land-monthly-means?tab=form'>Climate Copernicus website</a>. Check the following options:
</p>


<table border="1" style="margin-top: 20px; margin-bottom: 20px; border-collapse: collapse; width: 100%;">
    <tr>
        <th style="padding: 10px; text-align: left;">Field</th>
        <th style="padding: 10px; text-align: left;">Details</th>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Product type:</strong></td>
        <td style="padding: 10px;"><em>Monthly averaged reanalysis</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Variable:</strong></td>
        <td style="padding: 10px;">
            <ul style="margin: 0; padding-left: 20px;">
                <li><em>2m temperature</em> (t2m)</li>
                <li><em>Total precipitation</em> (tp)</li>
                <li><em>Surface latent heat flux</em> (slhf)</li>
                <li><em>Surface sensible heat flux</em> (sshf)</li>
                <li><em>Surface solar radiation downwards</em> (ssrd)</li>
                <li><em>Forecast albedo</em> (fal)</li>
                <li><em>Surface net thermal radiation</em> (str)</li>
                <li>...</li>
                <li>You can explore additional options as you prefer. In this example, we use the variables listed above.</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Year:</strong></td>
        <td style="padding: 10px;"><em>Select all</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Month:</strong></td>
        <td style="padding: 10px;"><em>Select all</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Time:</strong></td>
        <td style="padding: 10px;"><em>Select all</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Geographical area:</strong></td>
        <td style="padding: 10px;">
            <em>Either download for the entire Earth, or specify the coordinates for a bounding box in the region of interest. For the region of interest, provide a North, East, South, and West coordinate, specifying the two corners of the bounding box.</em>
        </td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Format:</strong></td>
        <td style="padding: 10px;"><em>NetCDF-3</em></td>
    </tr>
</table>


<p style='text-align: justify;'>
Then click <i>Submit Request</i> (after you have registered or logged into your account). Please be aware that downloading this data can take up to several hours, depending on the number of variables, timescales, and the area selected. Once the download is complete, place the netCDF file in the correct directory and rename it accordingly.
</p>
<p style='text-align: justify;'>
Additionally, we need the _geopotential height_ as an extra variable in our dataset. We will calculate a new variable by determining the difference between the geopotential height and the elevation at the stake measurement. The purpose of this height difference is to encourage the model to use it for downscaling the ERA5Land climate data, rather than relying on the lapse rate. The geopotential is downloaded from <a href='https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation#ERA5Land:datadocumentation-parameterlistingParameterlistings'>here</a>, and is already included in this example.
</p>   

In [7]:
# Specify the files of the climate data, that will be matched with the coordinates of the stake data
if example:
	climate_data = './example_data/iceland/climate/era5_monthly_averaged_data.nc'
	geopotential_data = './example_data/iceland/climate/era5_geopotential_pressure.nc'
else:
	if era5carra:
		climate_data = './example_data/svalbard/era5_monthly_Svalbard.nc'
		geopotential_data = './example_data/iceland/climate/era5_geopotential_pressure.nc'
	else:
		geopotential_data = './example_data/svalbard/carra_east_geopotential_pressure.nc'
		climate_data = './example_data/svalbard/carra_monthly_east_rotatedgrid.nc'


#dataset.data.drop(dataset.data.tail(3000).index, inplace = True)

# Match the climate features, from the ERA5Land netCDF file, for each of the stake measurement dataset
dataset.get_climate_features(climate_data=climate_data, geopotential_data=geopotential_data,era5=era5carra)


<h2>4. Transform Data to Monthly Time Resolution</h2>
<p style='text-align: justify;'>
Finally, we need to transform the dataset into a monthly resolution, accounting for a variable period for the SMB target data. This will be done in order to perform SMB predictions at a monthly time step, which then will be integrated both in time and space to match the available glaciological and geodetic SMB observations for the training. Please specify the climate variables used in the previous step in the list below. Consult the <a href='https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation'>documentation</a> of all short names of the climate variables.
</p>

In [8]:
# Specify the short names of the climate variables available in the dataset
if era5carra:
	vois_climate = ['t2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str'] # era5
else:
	vois_climate = ['t2m', 'tp', 'slhf', 'sshf', 'ssrd', 'al', 'str'] # carra

# For each record, convert to a monthly time resolution
dataset.convert_to_monthly(vois_climate=vois_climate, vois_topographical=voi_topographical)

Finally, we can take a look at the dataset which will be used for training.

In [9]:
display(dataset.data)

,YEAR,POINT_LON,POINT_LAT,POINT_BALANCE,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE,RGIId,POINT_ID,ID,N_MONTHS,MONTHS,aspect,slope,t2m,tp,slhf,sshf,ssrd,fal,str
0,1991,11.827670,78.904665,-1430.0,82.090782,-58.909218,RGI60-07.00504,NaN,0,4,may,4.623422,0.259821,267.856024,0.001392,-9.593677e+05,-559318.233684,1.853156e+07,0.574771,-3.909071e+06
1,1991,11.827670,78.904665,-1430.0,82.090782,-58.909218,RGI60-07.00504,NaN,0,4,jun,4.623422,0.259821,273.593038,0.000565,-3.107870e+05,114957.494270,2.445124e+07,0.489040,-4.364667e+06
2,1991,11.827670,78.904665,-1430.0,82.090782,-58.909218,RGI60-07.00504,NaN,0,4,jul,4.623422,0.259821,275.069123,0.001972,-3.363925e+05,-38719.304900,1.917525e+07,0.484705,-3.034467e+06
3,1991,11.827670,78.904665,-1430.0,82.090782,-58.909218,RGI60-07.00504,NaN,0,4,aug,4.623422,0.259821,275.730792,0.002665,-1.427171e+05,604020.141791,1.000037e+07,0.482865,-1.769727e+06
4,1991,11.827670,78.904665,-1430.0,82.090782,-58.909218,RGI60-07.00504,NaN,0,4,sep,4.623422,0.259821,271.649688,0.000803,-1.098291e+06,-366596.514855,4.603186e+06,0.482853,-4.172610e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16919,2018,17.453729,78.811326,550.0,963.213711,-169.786289,RGI60-07.00570,NaN,2809,6,dec,1.779072,0.120950,262.739021,0.003992,-6.127001e+04,41873.413883,-1.912331e-01,0.849217,-2.103821e+06
16920,2018,17.453729,78.811326,550.0,963.213711,-169.786289,RGI60-07.00570,NaN,2809,6,jan,1.779072,0.120950,261.590647,0.002210,2.998347e+04,131476.498741,-1.912331e-01,0.849205,-2.156776e+06
16921,2018,17.453729,78.811326,550.0,963.213711,-169.786289,RGI60-07.00570,NaN,2809,6,feb,1.779072,0.120950,261.209475,0.003056,2.481746e+05,585999.409641,5.630498e+04,0.849265,-2.171752e+06
16922,2018,17.453729,78.811326,550.0,963.213711,-169.786289,RGI60-07.00570,NaN,2809,6,mar,1.779072,0.120950,253.043794,0.001112,1.209646e+05,614532.235545,3.159293e+06,0.849289,-3.042341e+06


<p style='text-align: justify;'>
We have finished preprocessing the training data for our machine learning model. You can either explore the dataset in this <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/date_exploration.ipynb'>notebook</a> or continue with model training in this <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/xgboost_model_training.ipynb'>notebook</a>.
</p>